In [146]:
import pandas as pd
import pickle
import os
import numpy as np
import random
from colog.colog import colog
c = colog()

In [147]:
def load_data_one_stock(path):
 
  file = open(path, 'rb')
  load_data= pickle.load(file)
  file.close()

  return load_data
  

In [148]:
# Settings 
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 1000
commission_value = 0.002
trainig_part = 0.5
test_part = 0.5

Exlude companies with bad ration and retraining!!!

Currten best settings:
{'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10}
-3458.0764813327787 495


In [149]:
## Load data

# file_name = 'df_stock_AAPL_period2y_interval1h.pkl'
# path = os.path.join(parent_path, folder_saved_models, file_name)
## stock_name = item[str.find(item, 'df_') + 3 : str.find(item, '_20')]
# df = load_data_one_stock(path)
## df.plot()
## print(df)

In [150]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
 
  if gmax == reference_point:
    result = False
  elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
    result = True
  else:
    result = False

  return result

def number_red_candles(i, k=11):

  if i < k:
    number_red_candles = (df['ha_colour'][0 : i] == 'red').sum()
  else:
    number_red_candles = (df['ha_colour'][i - k : i] == 'red').sum()
  return number_red_candles
    

In [167]:
def one_stock_simulation(df, ticker, mode, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test

    Returns:
      profits, wins, loses 
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'number wins': 0,
                'proportion': 0,
                'profit': 0,
                'transactions': [] }  
  last_top = df['close'].iloc[0]
  last_top_time = df['close'].index[0]
  last_top_i = 0
  reverse_flag = False
  sell_finished = True
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'full': 
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'test':
    range_ = range(int(df.shape[0] * trainig_part) + 1, df.shape[0] - 1)

  for i in range_:

    # dynaminc profit/loose coefficient
    if i > 200:
      if df['close'].iloc[i] / df['close'].iloc[i - 200] > bull_trend_coef: # bull trend
        gain_coef = 1.02
        lose_coef = 0.98  # not equal to 1 - k (DO Different Expertiments)
      else: # bear trend
        gain_coef = 1.005
        lose_coef = 0.95
    else: # undefined trend
      gain_coef = 1.02
      lose_coef = 0.98

    if sell_finished == False and df['high'].iloc[i] / buy_price > gain_coef:
      sell_price = float(df['high'].iloc[i])
      history_log['number success'] += 1
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number wins'] += 1
      history_log['transactions'].append({
        'type': 'success',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2), 
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['high'].index[i]
     })
      sell_finished = True

    if sell_finished == False and df['low'].iloc[i] / buy_price < lose_coef :
      sell_price = float(df['low'].iloc[i])
      sell_finished = True
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number fails'] += 1
      history_log['transactions'].append({
        'type': 'fail',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2),
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['low'].index[i]
      })

    # KEY parameters: RIV, prt, last_top_ratio
    # buy_time_cond = (df.index[i]).hour <= 15 and (df.index[i]).hour >= 11

    buy_ratio = float(last_top -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])
    if df['ha_pct'].iloc[i] > RIV \
      and reverse_flag==False \
      and sell_finished == True \
      and last_top / df['open'].iloc[i] > last_top_ratio \
      and i - last_top_i > distance_from_last_top \
      and buy_time_cond \
      and buy_ratio > buy_ratio_border \
      and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)) \
      and number_red_candles(i) > 6:

      reverse_flag = True
      # buy_price = float(df['close'].iloc[i])
      buy_price = float(df['open'].iloc[i + 1])
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      sell_finished = False

    # last top and reverse flag
    if df['ha_colour'].iloc[i] == 'red' \
      and df['ha_colour'].iloc[i - 1] == 'green'\
      and df['ha_colour'].iloc[i - 2] == 'green'\
      and df['ha_pct'].iloc[i - 1] > 0.1 \
      and df['ha_pct'].iloc[i - 2] > 0.1:
  
      last_top = df['high'].iloc[i - 1]
      last_top_i = i - 1
      reverse_flag = False
  
  history_log['proportion'] =   history_log['number wins'] / ( history_log['number fails'] + 1)

  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(history_log)
  
  if print_trans_history:
    for tran in history_log['transactions']:
      print(tran)

  return history_log['profit'], history_log['number wins'], history_log['number fails']


In [152]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12

best_params = {'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}

for param in best_params:
  vars()[param] = best_params[param]

parameters_list_search = {
  'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100],
  'distance_from_last_top': [0,2,3,5],
  'last_top_ratio': [1, 1.001, 1.002, 1.003],
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
  'buy_ratio_border' : [0,3,5,7,9,10],
  'bull_trend_coef': [1.12]
}

# parameters_list_search = {
#   'bull_trend_coef': np.arange(1.08, 1.25, 0.01)
# }

In [161]:
best_profit = -10**5
best_profit_improved = True
optimize_profit = True
ticker_stats = {}
stock_name_list_opt = stock_name_list
# for parameter in parameters_list_search:
while best_profit_improved:
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    for value in parameters_list_search[parameter]:
      vars()[parameter] = value
      c.print(f'parametr is {parameter} value is {value}:', color='white')
      total_profit = 0
      total_loses = 0
      total_wins = 0
      for ticker in stock_name_list_opt:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        profit, wins, loses= one_stock_simulation(df, ticker, mode='train')
        total_profit += profit
        total_loses += loses
        total_wins += wins
        ticker_stats[ticker] = {
          'profit': profit,
          'wins': wins,
          'loses' : loses,
        }
      
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}', color = 'red')
    c.print(f'total_profit_list: {total_profit_list}', color='orange')
    if optimize_profit:
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
    else:
      vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    # np.max(total_profit_list), np.argmax(total_profit_list)

  parameters_list_best = {}
  for parameter in parameters_list_search:
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    
    parameters_list_best[parameter] = vars()[parameter]
    
  print(parameters_list_best)
  print(max(total_profit_list), min(total_loses_list))

  best_profit_improved = max(total_profit_list) > best_profit
  best_profit = max(total_profit_list) if best_profit_improved else best_profit

  # Optimization which stocks to use:
  for ticker in stock_name_list_opt:
    if ticker_stats[ticker]['wins'] / (ticker_stats[ticker]['loses'] + 1) < 3 \
        or ticker_stats[ticker]['profit'] < 0:
      stock_name_list_opt.remove(ticker)
  c.print(f'Current optimal stock list len is {len(stock_name_list_opt)}, list is {stock_name_list_opt}', color='green')


parametr is last_top_ratio value is 1:
Total Profit is -2248.97, total loses is 1263
parametr is last_top_ratio value is 1.001:
Total Profit is -2088.81, total loses is 1254
parametr is last_top_ratio value is 1.002:
Total Profit is -2288.40, total loses is 1251
parametr is last_top_ratio value is 1.003:
Total Profit is -2210.44, total loses is 1242
total_profit_list: [-2248.9654696903226, -2088.8130666255947, -2288.4025565052034, -2210.4442467975605]
parameter last_top_ratio best value is 1.001
parametr is distance_from_last_top value is 0:
Total Profit is -1881.68, total loses is 1268
parametr is distance_from_last_top value is 2:
Total Profit is -2088.81, total loses is 1254
parametr is distance_from_last_top value is 3:
Total Profit is -2447.58, total loses is 1227
parametr is distance_from_last_top value is 5:
Total Profit is -3287.26, total loses is 1160
total_profit_list: [-1881.684818725586, -2088.8130666255947, -2447.582384622575, -3287.2649554443365]
parameter distance_from_l

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
6056.55094263172 1011
Current optimal stock list len is 127

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
7049.139814422607 599
Current optimal stock list len is 99,

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 10, 'bull_trend_coef': 1.12}
8616.535972952845 423
Current optimal stock list len is 97

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
8455.558336365704 662
Current optimal stock list len is 71

v1:
{'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
5762.099450752256 170 , number_red_candles= 6

In [72]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)

191

In [182]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

# Current_optimal_stock_list 
stock_name_list_opt = ['JPM', 'UNH', 'AVGO', 'HD', 'PEP', 'COST', 'CRM', 'WMT',
                                'MCD', 'ABT', 'TXN', 'WFC', 'INTC', 'RTX', 'HON', 'SPGI',
                                  'BA', 'AMGN', 'AMAT', 'SBUX', 'PLD', 'MS', 'DE', 'AXP', 
                                  'ADI', 'TJX', 'MMC', 'CVS', 'SCHW', 'TMUS', 'ETN', 'CI', 
                                  'BSX', 'AON', 'ITW', 'SNPS', 'KLAC', 'CDNS', 'WM', 'HCA',
                                    'FDX', 'ORLY', 'MMM', 'CMG', 'GM', 'MCK', 'NXPI', 'NSC',
                                      'HUM', 'DXCM', 'PH', 'MPC', 'ROP', 'MCHP', 'MSI', 'GD',
                                        'JCI', 'SRE', 'ADSK', 'AZO', 'TDG', 'AJG', 'TT', 'PCAR',
                                          'COF', 'CTAS', 'AIG', 'ANET', 'SPG', 'MSCI', 'ROK']

len(stock_name_list_opt), len(stock_name_list)


parameter is_near_global_max_prt best value is 80
parameter distance_from_last_top best value is 0
parameter last_top_ratio best value is 1
parameter RIV best value is 0.25
parameter buy_ratio_border best value is 9
parameter bull_trend_coef best value is 1.12


(71, 79)

In [169]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
print("TRAIN RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  profit, wins, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=False)
  total_profit += profit
  total_loses += loses
  total_wins += wins
print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_loses / total_wins + 1:.2f}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  profit, wins, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=True)
  total_profit += profit
  total_loses += loses
  total_wins += wins
print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_loses / total_wins + 1:.2f}')


TRAIN RESULTS
--------------------------------------------------
Simulation results for ticker GOOG:
{'number success': 13, 'number fails': 3, 'number wins': 13, 'proportion': 3.25, 'profit': 5.011505920410151, 'transactions': [{'type': 'fail', 'buy_price': 110.92, 'sell_price': 107.64, 'ratio': 23.77, 'buy_time': Timestamp('2022-06-30 11:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-01 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 111.63, 'sell_price': 114.0, 'ratio': 25.73, 'buy_time': Timestamp('2022-07-14 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-15 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 118.01, 'sell_price': 120.55, 'ratio': 10.83, 'buy_time': Timestamp('2022-08-09 14:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-08-10 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 115.42, 'sell_price': 117.06, 'ratio': 18.63, 'buy_time'